In [1]:
import numpy as np   # Importo paquete de álgebra lineal
import matplotlib.pyplot as plt   # Importo paquete de gráficos
import copy
import mef
plt.rc('figure', figsize=(15,8))   # Para gráficos
plt.rc('font',size=22)             # Para gráficos

# Punto 4

Determine los desplazamientos y rotaciones y fuerzas y torques de vínculos para el sistema de la figura. Tome $E = 210\: GPa$ e $I = 2 x 10^{–4}\: m^4$.
<center><img src="ImgProb04.png"></center>

In [2]:
# Defino la matriz de nodos "MN".
# - Columna 1 es la coordenada "x" del nodo.
# - Columna 2 es la coordenada "y" del nodo.
# - Columna 3 es la coordenada "z" del nodo.
MN = np.array([[0,0,0],
               [3,0,0],
               [6,0,0],
               [6,-1,0]])   # Resorte, le puse "-1" porque va para abajo, , pero NO IMPORTA porque ya tengo la constante "k" del resorte.

# Su número de filas es el número de NODOS "Nn".
Nn = MN.shape[0]

In [3]:
# Defino la matriz de conectividad "MC".
# Informa qué nodos componen a cada elemento.
MC= np.array([[0,1],
              [1,2],
              [2,3]])

# Su número de filas es el número de ELEMENTOS "Ne".
# Su número de columnas es el número NODOS POR ELEMENTO "Nnxe".
Ne, Nnxe = MC.shape

In [4]:
# Defino los grados de libertad por nodo "glxn".
glxn = 2

In [5]:
# Defino el módulo de elasticidad "E". Aplica para los elementos "0" y "1" (barras).
E = 210e9   # Pa

# Defino el momento de inercia "I". Aplica para los elementos "0" y "1".
I = 2e-4   # m^4

# Defino la longitud "L" de los elementos "0" y "1".
L = 3   # m

# Defino la constante "k" del elemento "2" (resorte).
k = 200e3 # N/m

### Condiciones de vínculo en DESPLAZAMIENTO y ROTACIÓN
- $y_0 = 0\: m$ (empotramiento)
- $\phi_0 = 0$
- $y_1 = 0\: m$
- $y_3 = 0\: m$ (empotramiento)
- $\phi_3 = 0$ (el resorte no puede rotar)

### Condiciones de vínculo en FUERZA y TORQUE
- $M_1 = 0\: kN$
- $F_2 = -50\: kN$
- $M_2 = 0\: kN$

In [6]:
# Defino vector "s" que contiene los nodos con condiciones de vínculo en desplazamiento.
s = np.array([0,1,2,6,7])

# Defino vector "Us" con los valores de las condiciones de vínculo.
Us = [[0],[0],[0],[0],[0]]

# Defino vector "r" que contiene los nodos con condiciones de vínculo en fuerza. Es el complemento de "s".
r = np.array([i for i in range(Nn*glxn) if i not in s])

# Defino vector "Fr" con los valores de las condiciones de vínculo.
Fr = [[0],[-50000],[0]]

## Resolución

In [7]:
archivo= 'Matrices_elementales.txt'
with open(archivo,'w') as f:   # Creo archivo desde cero, por eso uso "w".
    f.write('Matrices Elementales\n ===============')
archivo1= 'Matriz_global.txt'
with open(archivo1,'w') as f:   # Creo archivo desde cero, por eso uso "w".
    f.write('Matriz Global\n ===============')

In [8]:
# Defino la matriz elemental "Ke1" de los elementos "0" y "1".
Ke1 = (E*I/L**3)*np.array([[12,6*L,-12,6*L],
                           [6*L,4*(L**2),-6*L,2*(L**2)],
                           [-12,-6*L,12,-6*L],
                           [6*L,2*(L**2),-6*L,4*(L**2)]])

In [9]:
# Defino la matriz elemental "Ke2" del elemento "2". Al ser un resorte, debería ser de [k -k; -k k]. Sin embargo, ahora
# quiero que tenga tamaño "4*4" con desplazamientos "y" y ángulos "phi".
# - Las filas y columnas que tengan "phi" deben ser nulas, porque el resorte no se torsiona.
# - Las filas y columnas que tengan "y" deben corresponder a [k -k; -k k].
Ke2 = (k)*np.array([[1,0,-1,0],
                    [0,0,0,0],
                    [-1,0,1,0],
                    [0,0,0,0]])

In [10]:
# Defino matriz global "Kg".
Kg = np.zeros([glxn*Nn, glxn*Nn])

# Ensamblo las matrices elementales para obtener la matriz global "Kg".
for e in range(Ne):
    if e==0 or e==1:
        # El elemento es una barra.
        Ke = Ke1
    else:
        # El elemento es un resorte.
        Ke = Ke2
    fe = np.abs(Ke.max()) # Factor de escala, para que los números en "Ke" se lean mejor.
    with open(archivo,'a') as f:   # Voy reescribiendo el archivo con nuevas "Ke", por eso uso "a".
        f.write(f'\nelemento {e}, fe = {fe:4e}\n')
        f.write(f'{Ke/fe}\n')

    for i in range(Nnxe):
        rangoi = np.linspace(i*glxn, (i+1)*glxn-1, Nnxe).astype(int)
        rangoni = np.linspace(MC[e, i]*glxn, (MC[e, i]+1)*glxn-1, Nnxe).astype(int)
        for j in range(Nnxe):
            rangoj = np.linspace(j*glxn, (j+1)*glxn-1, Nnxe).astype(int)
            rangonj = np.linspace(MC[e, j]*glxn, (MC[e, j]+1)*glxn-1, Nnxe).astype(int)
            Kg[np.ix_(rangoni, rangonj)] += Ke[np.ix_(rangoi, rangoj)]
fe = np.abs(Kg.max())
with open(archivo1,'a') as f:   # Reescribo el archivo con la matriz global "Kg" obtenida, por eso uso "a".
    f.write(f'\nMatriz Global, fe = {fe:4e}\n')
    f.write(f'{Kg/fe}\n')

In [11]:
# Llamo al paquete "mef", que contiene la función "solve" que calcula los vectores de fuerzas y torques "F", y de 
# desplazamientos y rotaciones "U", empleando MEF. 
F, U = mef.solve(Kg, r, Fr, s, Us)

In [12]:
print('Las FUERZAS (en N) y TORQUES (en N.m) son:')
print(F)
print('Los DESPLAZAMIENTOS (en m) y ROTACIONES son:')
print(U)

Las FUERZAS (en N) y TORQUES (en N.m) son:
[[-69767.44186047]
 [-69767.44186047]
 [116279.06976744]
 [     0.        ]
 [-50000.        ]
 [     0.        ]
 [  3488.37209302]
 [     0.        ]]
Los DESPLAZAMIENTOS (en m) y ROTACIONES son:
[[ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [-0.00249169]
 [-0.01744186]
 [-0.00747508]
 [ 0.        ]
 [ 0.        ]]


In [13]:
# FORMA MÁS LINDA DE PRESENTAR LOS DATOS:
# - "%s" significa que te pone número entero.
# - "%.4f" significa que te pone número con 4 cifras decimales.
# - "%2.4f" y "%7.4f" sólo varían en que con "7" te pone los números alineados respecto del "=" y queda más lindo si 
# llega  a haber uno con signo "-".
for nodo in range(Nn):
    print('Nodo %s     Uy = %8.4f mm     Phi = %7.4f     Fy = %11.4f N     M_Phi = %11.4f N.m'%(nodo+1, U[2*nodo]*1000, U[2*nodo+1]*1000, F[2*nodo], F[2*nodo+1]))

Nodo 1     Uy =   0.0000 mm     Phi =  0.0000     Fy = -69767.4419 N     M_Phi = -69767.4419 N.m
Nodo 2     Uy =   0.0000 mm     Phi = -2.4917     Fy = 116279.0698 N     M_Phi =      0.0000 N.m
Nodo 3     Uy = -17.4419 mm     Phi = -7.4751     Fy = -50000.0000 N     M_Phi =      0.0000 N.m
Nodo 4     Uy =   0.0000 mm     Phi =  0.0000     Fy =   3488.3721 N     M_Phi =      0.0000 N.m
